### Importation de Bibliothèques 

In [1]:
import os
import sys 
sys.path.append('/home/onyxia/work/libsigma')
# Personal librairies
import read_and_write as rw
import classification as cla
import numpy as np
import matplotlib.pyplot as plt

### Définition des chemins de jeux de données 

In [2]:
my_folder = '/home/onyxia/work/Depot_Git/results/data'
sample_filename = os.path.join(my_folder, 'sample/Sample_BD_foret_T31TCJ.tif')
image_filename = os.path.join(my_folder, 'img_pretraitees/Serie_temp_S2_ndvi.tif')

### Extraction de classes  à partir de fichier des échantillons et des valeurs de pixels à partir de raster de NDVI 

In [ ]:
X, Y, t = cla.get_samples_from_roi(image_filename, sample_filename)
print(X.shape)
print(Y.shape)
# Vérification de valeur min et max de NDVI 
min_X = X.min()
max_X = X.max()  
print(f"Valeur minimale  de X: {min_X}")
print(f"Valeur maximale de X: {max_X}")

# Vérifiaction s'il y a des valeur de NDVI en dehors de l'intervalle [-1,1]
out_of_range = X[(X < -1) | (X > 1)] 
if len(out_of_range) > 0:
    print(f"valeurs de NDVI en dehors de l'intervalle [-1,1] : {out_of_range}")
else:
    print("Pas de valeur de NDVI en dehors de l'intervalle [-1,1]")

In [ ]:
# Convertion de matrice de label dans la bonne format 
Y = Y.flatten()  # Convierte Y de (4997108, 1) a (4997108,)

# Vérification de la taille de matrice de labels et la matrice des valeurs de NDVI pour le six dates 
print(X.shape)  # (4997108, 6)
print(Y.shape)  # (4997108,)

# list de codes de classes d'intérêt 
codes_of_interest = ['12', '13', '14', '23', '24', '25']

# Filtrage pour ne garder que les pixels dnt la code de la classe est inclu dans la liste de codes d'intérêt
mask = np.isin(Y.astype(str), codes_of_interest) 

X_filtered = X[mask]  
Y_filtered = Y[mask] 

# Vérification de résultat de filtrage appliqué
print("Shape de X_filtered:", X_filtered.shape)
print("Shape de Y_filtered:", Y_filtered.shape)

# Vérification des classes présentes après filtrage 
unique_classes_in_y = np.unique(Y)
print(f"Classes uniques : {unique_classes_in_y}")


### Visualisation graphique de l'évolution de signature Spectrale de moyenne et l'écart-type par classe en fonction de dates  

In [ ]:
# Liste de dates 
dates = ['2023-01-01', '2023-03-01', '2023-05-01', '2023-07-01', '2023-09-01', '2023-11-01']

# Coleurs choisis pour chaque classe 
colors = ['tan', 'palegreen', 'limegreen', 'darkgreen', 'lightblue', 'orange']
# Liste de Labels 
class_names = ['Chêne', 'Robinier', 'Peupleraie', 'Douglas', 'Pin laricio ou pin noir', 'Pin maritime']

# Préparation de fênetre graphique 
fig, ax = plt.subplots(figsize=(10, 6))

# Parcourir la liste de code de classes et calculer pour chaque classse la valeur moyenne et l'écart-type pour chaque date 
for idx, code in enumerate(codes_of_interest):
    X_class = X_filtered[Y_filtered == code]
    
    if X_class.shape[0] > 0:
        # Calcul de moyenne et d'écart-type de NDVI 
        means = X_class.mean(axis=0)
        stds = X_class.std(axis=0)
        
        ax.plot(dates, means, color=colors[idx], label=class_names[idx])
        ax.fill_between(dates, means - stds, means + stds, color=colors[idx], alpha=0.3)

# Configurer les étiquettes et le graphique 
ax.set_xlabel('Dates')
ax.set_ylabel('NDVI')
ax.set_title('Evolution de la signature spectrale par classe en fonction de dates')

ax.set_ylim(0, 1)

ax.legend(title="Classes", loc='upper left')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()